In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Reconstructing the data from the text provided
data = {
    32: {
        'iters': [72584, 36292, 18146, 9073, 4537, 2269, 1135, 757, 568, 379, 284, 190, 142, 95, 71, 48, 36],
        'time': [29.060738, 14.59808, 7.305887, 3.612627, 1.799305, 0.904532, 0.912695, 0.915172, 0.918397, 0.91716, 0.919556, 0.920039, 0.917436, 0.909852, 0.902546, 0.911472, 0.913548]
    },
    64: {
        'iters': [36292, 18146, 9073, 4537, 2269, 1135, 568, 379, 284, 190, 142, 95, 71, 48, 36, 24, 18],
        'time': [14.846445, 7.457228, 3.704672, 1.836223, 0.91716, 0.475239, 0.478634, 0.479281, 0.481196, 0.48144, 0.485448, 0.484478, 0.487195, 0.486387, 0.486524, 0.490529, 0.496619]
    },
    128: {
        'iters': [18146, 9073, 4537, 2269, 1135, 568, 284, 190, 142, 95, 71, 48, 36, 24, 18, 12, 9],
        'time': [7.524805, 3.746993, 1.850193, 0.920569, 0.474645, 0.47805, 0.480579, 0.481035, 0.484991, 0.48425, 0.488036, 0.487391, 0.487055, 0.490689, 0.496299, 0.505625, 0.518298]
    },
    256: {
        'iters': [9073, 4537, 2269, 1135, 568, 284, 142, 95, 71, 48, 36, 24, 18, 12, 9, 6, 5],
        'time': [3.745461, 1.848435, 0.918989, 0.474693, 0.47869, 0.481672, 0.48719, 0.486761, 0.491201, 0.490909, 0.491016, 0.495231, 0.501145, 0.511099, 0.524333, 0.546403, 0.570066]
    },
    512: {
        'iters': [4537, 2269, 1135, 568, 284, 142, 71, 48, 36, 24, 18, 12, 9, 6, 5, 3, 3],
        'time': [1.853778, 0.919937, 0.474679, 0.479049, 0.482904, 0.489123, 0.495183, 0.496091, 0.497108, 0.502915, 0.510292, 0.522325, 0.537908, 0.563052, 0.58982, 0.63916, 0.69215]
    },
    1024: {
        'iters': [2269, 1135, 568, 284, 142, 71, 36, 24, 18, 12, 9, 6, 5, 3, 3, 2, 2],
        'time': [0.921125, 0.475732, 0.479818, 0.484177, 0.491396, 0.499491, 0.504061, 0.51169, 0.520879, 0.535869, 0.554594, 0.584297, 0.61437, 0.674252, 0.733246, 0.852065, 0.971631]
    }
}

# 1. Main Analysis Plot (Log-Log Scale)
plt.figure(figsize=(12, 7))
colors = sns.color_palette("viridis", len(data))

for i, (block_size, values) in enumerate(data.items()):
    plt.plot(values['iters'], values['time'], marker='o', label=f'Block Size {block_size}', color=colors[i], markersize=4)

plt.xscale('log')
plt.yscale('log')
plt.gca().invert_xaxis() # High iterations (left) -> Low iterations (right) to show "Progress" of parallelization

plt.xlabel('Grid-Stride Loop Iterations (Log Scale)\n<-- More Serial (High Load/Thread) | More Parallel (Low Load/Thread) -->', fontsize=11)
plt.ylabel('Execution Time (ms) [Log Scale]', fontsize=11)
plt.title('Impact of Per-Thread Workload on Performance\n(Bathtub Curve Analysis)', fontsize=14)
plt.legend(title="Block Config")
plt.grid(True, which="both", ls="--", alpha=0.4)

# Add annotations for phases
plt.axvline(x=1000, color='gray', linestyle=':', alpha=0.5)
plt.text(20000, 10, 'Phase 1: Under-utilization\n(Not enough threads)', fontsize=10, color='red')
plt.text(800, 0.35, 'Phase 2: Sweet Spot\n(Balance)', fontsize=10, color='green', ha='center')
plt.text(5, 1.2, 'Phase 3: Overhead/Contention\n(Too many threads)', fontsize=10, color='orange')

plt.tight_layout()
plt.savefig('01_iterations_analysis_log.png')
plt.close()

# 2. Zoomed-in Plot (Linear Scale for Time, Log Scale for X)
# Focused on the "bottom of the bathtub" to see the hook effect
plt.figure(figsize=(12, 6))

for i, (block_size, values) in enumerate(data.items()):
    # Filter only data where time < 2.0ms to focus on the sweet spot
    filtered_iters = [x for x, y in zip(values['iters'], values['time']) if y < 1.0]
    filtered_time = [y for y in values['time'] if y < 1.0]
    if filtered_iters:
        plt.plot(filtered_iters, filtered_time, marker='o', label=f'Block Size {block_size}', color=colors[i], markersize=5)

plt.xscale('log')
# plt.yscale('linear') # Linear is better here to see the small degradation
plt.gca().invert_xaxis()

plt.xlabel('Grid-Stride Loop Iterations (Log Scale)', fontsize=11)
plt.ylabel('Execution Time (ms)', fontsize=11)
plt.title('Detail View: The "Tail Effect" (Over-Parallelization)', fontsize=14)
plt.legend(title="Block Config")
plt.grid(True, which="both", ls="--", alpha=0.4)

plt.tight_layout()
plt.savefig('01_iterations_analysis_zoom.png')
plt.close()